# Ответы на вопросы 1 задания.
1) .Признаки , имеющие пропуски (12 штук): 
first_blood_time,
first_blood_team ,
first_blood_player1 ,
first_blood_player2 ,
radiant_bottle_time ,
radiant_courier_time ,
radiant_flying_courier_time ,
radiant_first_ward_time ,
dire_bottle_time ,
dire_courier_time ,
dire_flying_courier_time ,
dire_first_ward_time ,
очевидно, эти события не произошли в первые 5 минут матча.

2). Целевая перемеменная: 'radiant_win'

3). Время для 30 деревьев  0:00:56.633813 , среднее roc_auc= 0.6897242180946103

4). Время для 40 деревьев  0:01:18.083617 40 среднее roc_auc= 0.6947062526423649, больше  увеличивать количество деревьев смысла нет.  Для ускорения обучения можно сократить число блоков  кросс валидации до 4-х и убрать категориальные признаки.

# Ответы на вопросы 2 задания
1).Логистическая регрессия над всеми исходными признаками работает быстрее (10 секунд против  примерно минуты)
время 0:00:10.206909, параметр C=  1, среднее roc_auc= 0.7163375761461741. Для градиентного бустинга  лучшее значение: время 0:01:29.176063, количество деревьев 45, среднее roc_auc= 0.6966472353314292. Почему у логистической регрессии над всеми исходными признаками получилось качество лучше  не знаю ((.

2). Удаление категориальных признаков  на качесто практически не повлияло. Новое лучшее значение среднее roc_auc= 0.7166383619896922  так же при С=1. Объяснить я это могу тем что необработанные категориальные признаки ничего не давали в качество и соответственно удаление их только чуть увеличило скорость,  качество при этом  почти не  поменялось.

3). Всего существует 108 разных идентификаторов игроков.

4). После добавления "мешка слов" : время  0:00:20.376371  при C= 100000000, среднее roc_auc= 0.751793057543279, качество заметно увеличилось вследствие использования дополнительной информации по игрокам.

5). Максимальное значение прогноза  выиграша radiant на тестовой выборке при C=1 получилось 0.9965118770761712,
минимальное  0.008613834853999164

In [2]:
import numpy as np
import pandas as pd

In [3]:
# загрузка файлов features, features_test
fe = pd.read_csv('features.csv', index_col='match_id')
fet=pd.read_csv('features_test.csv',index_col='match_id')
fe # посмотреть начало

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63
5,1430284186,1,11,5,1961,1461,19,0,1,6,...,4,4,0,-43.0,1453,0,512,2038,0,63
8,1430293701,1,8,3,967,1136,7,1,0,8,...,6,3,0,10.0,1968,0,1536,1983,12,63
9,1430299335,7,35,5,2117,1252,16,0,0,6,...,3,4,0,-15.0,4079,1,1540,0,63,0
11,1430308974,1,17,5,1527,906,10,0,1,7,...,3,4,0,26.0,3071,0,0,1572,0,63


In [30]:
# определение признаков  с неопределёнными значениями, чуть по другому  чем в инструкции, но работает...
n0t=pd.notna(fe)#  пометка False тех значений , которые не определены
for i in list(fe):# для каждого столбца  
    # определяем число значений = False (не определённых)
    if len(n0t.loc[n0t[i]==False])!=0:# проверяем  чтобы число таких значений не равно 0
        print i # и печатаем  имя  этого столбца

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


In [4]:
list(fe)#  список всех признаков

['start_time',
 'lobby_type',
 'r1_hero',
 'r1_level',
 'r1_xp',
 'r1_gold',
 'r1_lh',
 'r1_kills',
 'r1_deaths',
 'r1_items',
 'r2_hero',
 'r2_level',
 'r2_xp',
 'r2_gold',
 'r2_lh',
 'r2_kills',
 'r2_deaths',
 'r2_items',
 'r3_hero',
 'r3_level',
 'r3_xp',
 'r3_gold',
 'r3_lh',
 'r3_kills',
 'r3_deaths',
 'r3_items',
 'r4_hero',
 'r4_level',
 'r4_xp',
 'r4_gold',
 'r4_lh',
 'r4_kills',
 'r4_deaths',
 'r4_items',
 'r5_hero',
 'r5_level',
 'r5_xp',
 'r5_gold',
 'r5_lh',
 'r5_kills',
 'r5_deaths',
 'r5_items',
 'd1_hero',
 'd1_level',
 'd1_xp',
 'd1_gold',
 'd1_lh',
 'd1_kills',
 'd1_deaths',
 'd1_items',
 'd2_hero',
 'd2_level',
 'd2_xp',
 'd2_gold',
 'd2_lh',
 'd2_kills',
 'd2_deaths',
 'd2_items',
 'd3_hero',
 'd3_level',
 'd3_xp',
 'd3_gold',
 'd3_lh',
 'd3_kills',
 'd3_deaths',
 'd3_items',
 'd4_hero',
 'd4_level',
 'd4_xp',
 'd4_gold',
 'd4_lh',
 'd4_kills',
 'd4_deaths',
 'd4_items',
 'd5_hero',
 'd5_level',
 'd5_xp',
 'd5_gold',
 'd5_lh',
 'd5_kills',
 'd5_deaths',
 'd5_items',


In [31]:
hero_D = np.unique(fe['d1_hero'])

In [32]:
len(hero_D)

108

In [33]:
# заглядывющие вперёд признаки, они последние
fe[[ 'duration',
 'radiant_win',
 'tower_status_radiant',
 'tower_status_dire',
 'barracks_status_radiant',
 'barracks_status_dire']].head()

,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,
0,2874,1,1796,0,51,0
1,2463,1,1974,0,63,1
2,2130,0,0,1830,0,63
3,1459,0,1920,2047,50,63
4,2449,0,4,1974,3,63


In [7]:
X=fe.iloc[:,0:102]# исключаем, заглядывающие вперёд признаки ( последние 6 из 108)
y=fe['radiant_win'] # вектор целевой переменной

# 1) градиентный бустинг " в лоб"

In [35]:
Xf=X.fillna(1e8)# для  градиентного бустинга заменяем неопределённые значения очень большим числом

In [52]:
from sklearn import ensemble
import time
import datetime
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# генератор разбиений для кросс валидации по 5 блокам с перемешиванием
kf = KFold(n_splits=5,shuffle=True)
#  градиентный бустинг для значений  количества  деревьев: 10,20,30,40,45
for n in [10,20,30,40,45]:  
    start_time = datetime.datetime.now()# измерение времени каждого прохода
    clf = ensemble.GradientBoostingClassifier(n_estimators=n,max_depth=6)
    cr_v=cross_val_score(clf , Xf, y,cv=kf, scoring="roc_auc") # метрика качества "roc_auc"
    print 'Time elapsed:', datetime.datetime.now() - start_time, n,"среднее roc_auc=",np.mean(cr_v)
   

Time elapsed: 0:01:33.705708 10 среднее roc_auc= 0.6823620311066069
Time elapsed: 0:02:39.409579 20 среднее roc_auc= 0.694719698025604
Time elapsed: 0:04:03.433308 30 среднее roc_auc= 0.7008965010963925
Time elapsed: 0:05:24.479555 40 среднее roc_auc= 0.70418137909413
Time elapsed: 0:06:14.117954 45 среднее roc_auc= 0.7058612995778083


# 2) Логистическая регрессия

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
Xf0=X.fillna(0)# для логист. регрессии неопределённые значения заменяем  нулём
# приводим признаки к одному масштабу
scaler = StandardScaler()
Xf0s=scaler.fit_transform(Xf0)# отмасштабированная матрица

/home/dimok/.local/lib/python2.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/dimok/.local/lib/python2.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [40]:
# проводим логистическую регрессию для всех исходных признаков для разных значений С
for c in [1e8,1e4,1e2,1,1e-4,1e-8]:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=c,random_state=241)
    cr_v=cross_val_score(clf , Xf0s, y,cv=kf, scoring="roc_auc")
    print 'Time elapsed:', datetime.datetime.now() - start_time,'C= ',c,'среднее roc_auc=',np.mean(cr_v)
 

Time elapsed: 0:00:10.744352 C=  100000000.0 среднее roc_auc= 0.7164949225896255
Time elapsed: 0:00:10.967956 C=  10000.0 среднее roc_auc= 0.7163950727931739
Time elapsed: 0:00:10.645006 C=  100.0 среднее roc_auc= 0.7164932667666513
Time elapsed: 0:00:10.592244 C=  1 среднее roc_auc= 0.7163558527472397
Time elapsed: 0:00:04.357117 C=  0.0001 среднее roc_auc= 0.7113027756717908
Time elapsed: 0:00:02.048375 C=  1e-08 среднее roc_auc= 0.6872286933586494


In [9]:
# убираем категориальные признаки
Xd=X.drop(['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero'],axis=1)
Xd0=Xd.fillna(0)# для логист. регрессии неопределённые значения заменяем  нулём
# приводим признаки к одному масштабу
scaler = StandardScaler()
Xd0s=scaler.fit_transform(Xd0)# отмасштабированная матрица

In [44]:
# проводим логистическую регрессию для разных значений С
for c in [1e8,1e4,1e2,1,1e-4,1e-8]:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=c,random_state=241 )
    cr_v=cross_val_score(clf , Xd0s, y,cv=kf, scoring="roc_auc")
    print 'Time elapsed:', datetime.datetime.now() - start_time,'C= ',c,'среднее roc_auc=',np.mean(cr_v)

Time elapsed: 0:00:09.567822 C=  100000000.0 среднее roc_auc= 0.7162838678631958
Time elapsed: 0:00:09.734474 C=  10000.0 среднее roc_auc= 0.7164034053289872
Time elapsed: 0:00:09.278211 C=  100.0 среднее roc_auc= 0.7163867329525813
Time elapsed: 0:00:09.629131 C=  1 среднее roc_auc= 0.7164177430908211
Time elapsed: 0:00:03.924922 C=  0.0001 среднее roc_auc= 0.7113003634906263
Time elapsed: 0:00:01.861806 C=  1e-08 среднее roc_auc= 0.6871355137975861


# таким образом почти ничего не изменилось

In [45]:
# найдём и напечатаем количество модификаций двух разных идентификаторов героев
print 'r1_hero=',len(fe['r1_hero'].value_counts()),'d4_hero=',len(fe['d4_hero'].value_counts())

r1_hero= 108 d4_hero= 108


In [4]:
# Используем приведённый  в задании код для формирования "мешка слов" по героям
X_pick = np.zeros((fe.shape[0], 112))#только непонятно почему работает  с числом N=112 и больше , но не со 108 ?
for i, match_id in enumerate(fe.index):
     for p in xrange(5):
        X_pick[i, fe.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, fe.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [10]:
# прибавляем новые признаки вместо удалённых категориальных
xj=np.concatenate((Xd0s,X_pick), axis=1)

In [48]:
# проводим логистическую регрессию для новой матрицы с "мешком слов"  для разных значений С
for c in [1e8,1e4,1e2,1,1e-4,1e-8]:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=c,random_state=241 )
    cr_v=cross_val_score(clf , xj, y,cv=kf, scoring="roc_auc")
    print 'Time elapsed:', datetime.datetime.now() - start_time,'C= ',c,'среднее roc_auc=',np.mean(cr_v)

Time elapsed: 0:00:20.438476 C=  100000000.0 среднее roc_auc= 0.7517880632187155
Time elapsed: 0:00:19.863625 C=  10000.0 среднее roc_auc= 0.751944407892817
Time elapsed: 0:00:20.232921 C=  100.0 среднее roc_auc= 0.7518030820417302
Time elapsed: 0:00:19.284912 C=  1 среднее roc_auc= 0.751948233394512
Time elapsed: 0:00:04.150635 C=  0.0001 среднее roc_auc= 0.7249111568232254
Time elapsed: 0:00:01.860347 C=  1e-08 среднее roc_auc= 0.6892236668168733


# Видим что качество заметно улучшилось.

In [49]:
# fet - тестовый набор
Xt0=fet.fillna(0)# для логист регрессии неопределённые значения заменяем  нулём
# удаляем категориальные признаки
Xt0d=Xt0.drop(['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero'],axis=1)

In [50]:
# формируем"мешок слов" для тестовой выборки
X_pick_t = np.zeros((fet.shape[0], 112))
for i, match_id in enumerate(Xt0d.index):
     for p in xrange(5):
        X_pick_t[i, fet.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_t[i, fet.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [51]:
Xt0ds=scaler.fit_transform(Xt0d)# масштабируем признаки
Xtj=np.concatenate((Xt0ds,X_pick_t), axis=1) # прибавляем "мешок слов"
clf = LogisticRegression(C=100,random_state=241 )#лучшее С=1
clf.fit(xj,y)# обучаем  классификатор на обучающей выборке 
prw=clf.predict_proba(Xtj)[:,1]# применяем к  отмасштабированной тестовой выборке с мешком слов
print np.max(prw),np.min(prw) # печатаем наибольшее и наименьшее значения  вероятностей 1-го класса(выигрыша radiant)

0.9965177266136428 0.008617507750173305


In [27]:
# формирование таблицы ответов
p=pd.DataFrame(prw)
ind=pd.DataFrame(fet.index)
t=ind.join(p)
t.columns = ['match_id', 'radiant_win']
t.to_csv('out.csv',index=False)# запись таблицы в файл csv под именем out.csv

In [28]:
# проверка что записалось
tt=pd.read_csv('out.csv')

In [29]:
tt.head(10)

,match_id,radiant_win
0,6,0.825673
1,7,0.758615
2,10,0.187050
3,13,0.861219
4,16,0.239747
5,18,0.378870
6,19,0.529575
7,24,0.565865
8,33,0.215237
9,37,0.673855
